# **Vision Transformer 실습**

## **데이터셋 준비**

#### *Kaggle 데이터셋 로드*
**Stanford Dogs Dataset**  
Number of categories: 120  
Number of images: 20,580

In [1]:
import kagglehub

path = kagglehub.dataset_download("jessicali9530/stanford-dogs-dataset")
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/stanford-dogs-dataset


In [2]:
import os
import shutil

root_path = "/kaggle/processed"
org_image_path = "/kaggle/input/stanford-dogs-dataset/images/Images/"
labels = os.listdir(org_image_path)

labels_cnt_list = []

for l in labels:
    labels_cnt_list.append(len(os.listdir(os.path.join(org_image_path, l))))

# /kaggle/input 폴더에 직접 접근 불가; 해당 폴더는 read only임
os.makedirs(os.path.join(root_path, 'data/'), exist_ok=True)
os.makedirs(os.path.join(root_path, 'data/train/'), exist_ok=True)
os.makedirs(os.path.join(root_path, 'data/val/'), exist_ok=True)

for l in labels:
    os.makedirs(os.path.join(root_path, 'data/train/', l), exist_ok=True)
    os.makedirs(os.path.join(root_path, 'data/val/', l), exist_ok=True)

train_img_list = []
train_label_list = []
val_img_list = []
val_label_list = []

for idx, l in enumerate(labels):
    num_train = int(labels_cnt_list[idx] * 0.85)
    tmp_image_name_list = os.listdir(os.path.join(org_image_path, l))
    for cnt, fname in enumerate(tmp_image_name_list):
        if cnt <= num_train:
          dst_path = os.path.join(root_path, 'data/train/', l, fname)
          shutil.copy(os.path.join(org_image_path, l, fname), dst_path)
          train_img_list.append(dst_path)
          train_label_list.append(idx)
        else:
          dst_path = os.path.join(root_path, 'data/val/', l, fname)
          shutil.copy(os.path.join(org_image_path, l, fname), dst_path)
          val_img_list.append(dst_path)
          val_label_list.append(idx)

#### *CV 알고리즘 PyTorch 구현 패키지 install*

In [3]:
!pip install timm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

#### *이미지 파일 불러오기 + 전처리 함수 정의*


In [4]:
import torch
from PIL import Image
from torchvision import models, transforms

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, img_list, label_list, phase, transform):
        self.img_list = img_list
        self.label_list = label_list
        self.phase = phase
        self.transform = transform

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, index):
        # img = self.img_list[index]

        img_path = self.img_list[index]
        img = Image.open(img_path).convert('RGB')

        transformed_img = self.transform(img, self.phase)
        label = self.label_list[index]

        return transformed_img, label

class MyTransform():
    def __init__(self, resize, mean, std):
        self.data_transform = {
            # train의 경우 이미지 전처리
            'train': transforms.Compose([
                # 데이터 증강을 위해 이미지를 랜덤하게 크롭 및 좌우 반전
                transforms.RandomResizedCrop(
                    (resize, resize), scale=(0.5, 1.0)),
                transforms.RandomHorizontalFlip(),
                # 이미지를 텐서로 변환
                transforms.ToTensor(),
                # 정규화
                transforms.Normalize(mean, std)
            ]),
            # validation의 경우 이미지 전처리
            'val': transforms.Compose([
                # 이미지 리사이즈
                transforms.Resize((resize, resize)),
                # 이미지를 텐서로 변환
                transforms.ToTensor(),
                # 정규화
                transforms.Normalize(mean, std)
            ])
        }

    def __call__(self, img, phase='train'):
        return self.data_transform[phase](img)

## **모델 학습**

#### *모델 학습을 위한 train & validation 셋 정의*

In [5]:
size = 224  # 이미지 resize 크기 224*224
# ImageNet 기준 평균 & 표준편차
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

# train 데이터 셋 생성, 전처리 함수 호출
train_dataset = MyDataset(img_list=train_img_list, label_list=train_label_list, phase="train", transform=MyTransform(size, mean, std))
# validation 데이터 셋 생성, 전처리 함수 호출
val_dataset = MyDataset(img_list=val_img_list, label_list=val_label_list, phase="val", transform=MyTransform(size, mean, std))

image_datasets = {'train' : train_dataset, 'val' : val_dataset}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

#### *데이터 로드*

In [6]:
batch_size = 32

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

#### *ViT 모델 가져오기*
이미지를 224*224 사이즈의 16개 패치로 나누는 ViT 모델을 가져옴

In [7]:
import timm

num_classes = 120
model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=num_classes)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

#### *Loss Function & Optimizer & Scheduler 정의*
Loss는 **CrossEntropy** 사용  
Optimizer는 **Adam** 사용  
**StepLR**로 epoch 마다 학습률 감소시킴

In [8]:
import torch.optim as optims
import torch.nn as nn
from torch.optim import lr_scheduler

criterion = nn.CrossEntropyLoss()
optimizer = optims.Adam(model.parameters(), lr=0.001, betas=(0.9,0.999))
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

#### *학습 loop*

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = model.to(device)

epochs = 10
best_acc = 0.0

for epoch in range(epochs):
    print("{}/{} epoch running now".format(epoch, epochs - 1))

    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders_dict[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        if phase == 'train':
            exp_lr_scheduler.step()

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

        # 모델 저장
        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            torch.save(model.state_dict(), './best_model.pth')

0/9 epoch running now
train Loss: 4.8335 Acc: 0.0142
val Loss: 4.7827 Acc: 0.0142
1/9 epoch running now
train Loss: 4.6876 Acc: 0.0204
val Loss: 4.6558 Acc: 0.0202
2/9 epoch running now
train Loss: 4.6693 Acc: 0.0206
val Loss: 4.6632 Acc: 0.0222
3/9 epoch running now
train Loss: 4.6282 Acc: 0.0237
val Loss: 4.6811 Acc: 0.0192
4/9 epoch running now
train Loss: 4.6361 Acc: 0.0216
val Loss: 4.6612 Acc: 0.0232
5/9 epoch running now
train Loss: 4.6178 Acc: 0.0243
val Loss: 4.6029 Acc: 0.0255
6/9 epoch running now
train Loss: 4.6158 Acc: 0.0259
val Loss: 4.6393 Acc: 0.0265
7/9 epoch running now
train Loss: 4.5520 Acc: 0.0310
val Loss: 4.5694 Acc: 0.0391
8/9 epoch running now
train Loss: 4.5178 Acc: 0.0363
val Loss: 4.5461 Acc: 0.0364
9/9 epoch running now
train Loss: 4.5003 Acc: 0.0387
val Loss: 4.5433 Acc: 0.0351


## **시각화**
학습 후 GPU 제한으로 시각화 코드 사용을 못했습니다..

In [10]:
from matplotlib import pyplot as plt

def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
visualize_model(model)